In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
import z3
from funcy import *

In [ ]:
from FV import *

In [ ]:
M = 3
MAX_OBJECT_SET_SIZE = 26
# M = 10
# MAX_OBJECT_SET_SIZE = 100
MAX_SAMPLE_SIZE = 8
# MAX_OBJECT_SET_SIZE = MAX_NUM_VECTORS
# MAX_SAMPLE_SIZE = MAX_NUM_VECTORS 
MAX_NUM_VECTORS = 3 ** M
N = randint(2, MAX_OBJECT_SET_SIZE)
assert N <= (3 ** M)
print(f"|features| = {M}")
print(f"=> |logically possible vectors| = {MAX_NUM_VECTORS}")
print(f"max |objects = inventory| = {MAX_OBJECT_SET_SIZE}")
print(f"max |sample = hypothetical natural class| = {MAX_SAMPLE_SIZE}")
print(f"actual |objects = inventory| = {N}")

In [ ]:
O = FeatureVector.randomObjects(numFeatures=M, numObjects=N)

if len(O) < 30:
  for each in O:
    print(each)

In [ ]:
# S = randomNonemptySubset(set(O), 3)
S = randomInterestingSubset(400000, set(O), cap=MAX_SAMPLE_SIZE, minSpec=2) # "interesting" = the natural class is something other the trivial zero vector [0 ... ]

if S is not None:
  for each in S:
    print(each)

  print("---")
  print(grandFoldl(FeatureVector.meet)(S))
else:
  print("S is None.")

In [ ]:
def notQuite_naturalClassFor_euf(featureVectors, solver=None):
  if solver is None:
    solver = z3.Solver()
  
  data = tuple(sorted(featureVectors, key=FeatureVector.sortKey))
  M    = len(data[0])
  
  defined = [z3.Bool(f"def_{i}") for i in range(M)]
  valence = [z3.Bool(f"val_{i}") for i in range(M)]
  
  for datum in data:
    solver.add(describes_euf(defined, valence, datum))
  
  if solver.check() == z3.sat:    
    print("Found model.")
    m = solver.model()
    return solver, m
  else:
    print("No model found.")
    return solver
  

def describes_euf(defined, valence, featureVector):
  # \forall i, defined[i] -> valence[i] == featureVector.value[i].valence()
  return z3.And([z3.Or([z3.Not(d), v == x.valence()])
                 for d,v,x in zip(defined, valence, featureVector.value)])
  

In [ ]:
naturalClassFor_euf(S)

In [ ]:
solver = z3.Solver()

for n in range(100):
  natural_class_solutions = SymbolicFV.mks(3, "NCs_S", n)
  SymbolicFV.naturalClasses(natural_class_solutions, list(SymbolicFV.mkFromConcreteFVs(S, "S")), O)
  if solver.check() == z3.sat:
    print(n)
    m = solver.model()